In [87]:
#%%
import pprint
import numpy as np 
import pandas as pd 
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
#%% hyperparameter
TRAIN_SIZE = 9000
# UpdateTime is an time string

In [3]:
#%% get tain_data and test_data
data = pd.read_csv("sample10000.csv").drop(labels="UpdateTime",axis=1).to_numpy()
labels = pd.read_csv("labels10000.csv").to_numpy()[:,0]

train_data = data[:TRAIN_SIZE]
test_data = data[TRAIN_SIZE:]

train_labels = labels[:TRAIN_SIZE]
test_labels = labels[TRAIN_SIZE:]

In [4]:
def model_score(model):
    #%% predict and score
    err = (test_labels -  model.predict(test_data))
    sqerr = (err*err).sum()
    # print(regr.get_params())
    #print("OOB score",regr.oob_score_)
    print("R^2 for train data :",model.score(train_data,train_labels))
    print("R^2 for test data :",model.score(test_data, test_labels))
    print("squre error:",sqerr)

In [5]:
#%% define model and train
def test_RF():
    regr = RandomForestRegressor(n_estimators=200,oob_score=True)
    regr.fit(train_data,train_labels)
    model_score(regr)
    # R^2 for train data : 0.8658618241928562
    # R^2 for test data : -0.07898389513463266
    # squre error: 313.67217500000004

In [79]:
#%% define DNN(MLPregressor)
def test_MLP():
    mlpr = MLPRegressor(hidden_layer_sizes=(10),
                        activation="logistic",batch_size=1000)
    # 激活函数很关键，relu 和 identity 基本不工作
    # batch_size 影响也很大
    mlpr.fit(train_data,train_labels)
    model_score(mlpr)

In [80]:
test_MLP()

R^2 for train data : -2.4562869375355234e-05
R^2 for test data : -0.0005819326087599386
squre error: 290.8798847529891


In [90]:

mlp_params = {
    "hidden_layer_sizes": [(x,) for x in range(10,150,10)],
    "activation":["logistic"],
    "batch_size":[1000,],
    "learning_rate":["adaptive"]
}
mlpr = MLPRegressor()
clf = GridSearchCV(mlpr,mlp_params,cv=10)
clf.fit(data,labels)
pprint.pprint(clf.cv_results_)
pprint.pprint(clf.best_params_)

{'mean_fit_time': array([0.30707519, 0.23105397, 0.31063137, 0.43048015, 0.41250656,
       0.477988  , 0.52048645, 0.53904328, 0.571492  , 0.58399563,
       0.67111053, 0.7416851 , 1.08477178, 1.01703644]),
 'mean_score_time': array([0.00077462, 0.00106084, 0.00173283, 0.00208986, 0.00177152,
       0.00248559, 0.00245175, 0.00242651, 0.00259323, 0.00318661,
       0.0032582 , 0.00409672, 0.00471044, 0.00490847]),
 'mean_test_score': array([-0.00026655, -0.00031599, -0.00039715, -0.00025771, -0.00073167,
       -0.00057833, -0.00056589, -0.00053924, -0.0006113 , -0.00105998,
       -0.00046324, -0.00275192, -0.00075188, -0.00032299]),
 'mean_train_score': array([-4.11983028e-05, -6.06222163e-06, -3.21406875e-05, -2.23295422e-05,
       -8.88309975e-05, -2.15716852e-04, -1.52501528e-04, -2.22184040e-04,
       -1.91552709e-04, -5.54882038e-04, -1.40495017e-04, -9.42724036e-04,
       -1.11916143e-04, -1.73176314e-04]),
 'param_activation': masked_array(data=['logistic', 'logistic', 'l